In [1]:
import pandas as pd
!pip install -U pandasql
from pandasql import sqldf

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=5650c908cc96ab5cadd26c21761db617b281d1e1210c7a48bc6b5e44158da025
  Stored in directory: /home/jovyan/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql


In [2]:
mysql = lambda q: sqldf(q, globals())
products= pd.read_csv('Amazon_Products.csv', low_memory=False)

In [3]:
prod_clean = products.loc[:, ~products.columns.str.contains('^Unnamed')]

In [41]:
### base of all analysis ~
cleaned_base = """
SELECT * FROM (
SELECT uniq_id
, product_name
, manufacturer
, price
, coalesce(cast(number_available_in_stock as int),0) number_available_in_stock
, coalesce(cast(number_of_reviews as int),0) number_of_reviews
, coalesce(cast(number_of_answered_questions as int),0) number_of_answered_questions
, cast(substr(average_review_rating,1,3) as float) avg_review_rating_num
, amazon_category_and_sub_category
, description
, product_information
, product_description
, items_customers_buy_after_viewing_this_item
, customer_questions_and_answers
FROM prod_clean)
WHERE number_of_reviews > 10
AND uniq_id is not null 
AND length(uniq_id) > 1
"""

category = """
SELECT distinct category_and_sub_category FROM (
SELECT uniq_id
, product_name
, manufacturer
, price
, coalesce(cast(number_available_in_stock as int),0) number_available_in_stock
, coalesce(cast(number_of_reviews as int),0) number_of_reviews
, coalesce(cast(number_of_answered_questions as int),0) number_of_answered_questions
, cast(substr(average_review_rating,1,3) as float) avg_review_rating_num
, coalesce(amazon_category_and_sub_category, 'None') category_and_sub_category
, description
, product_information
, product_description
, items_customers_buy_after_viewing_this_item
, customer_questions_and_answers
FROM prod_clean)
WHERE uniq_id is not null 
AND length(uniq_id) > 1
"""

In [42]:
mysql(category)

,category_and_sub_category
0,Hobbies > Model Trains & Railway Sets > Rail V...
1,Hobbies > Model Trains & Railway Sets > Lighti...
2,Hobbies > Model Trains & Railway Sets > Rail V...
3,Hobbies > Model Trains & Railway Sets > Lighti...
4,Hobbies > Model Trains & Railway Sets > Access...
...,...
251,Cooking & Dining > Kitchen Tools & Gadgets > B...
252,Arts & Crafts > Children's Craft Kits > Childr...
253,Novelty Jewellery > Charms > Bead Charms
254,Games > Travel & Pocket Games
